<a href="https://colab.research.google.com/github/chousheep/sat_simplification/blob/main/unsloth_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import json

file = json.load(open("batch_finetune.json", "r"))
print(file[1])

{'instruction': 'Translate into simple English.', 'input': 'When Alexa looked at the voluminous e-mail, she was tempted to abridge its contents.', 'output': 'When Alexa looked at the voluminous e-mail, she was tempted to shorten its contents.'}


# Install Dependencies

This cell installs the necessary libraries for fine-tuning the language model using Unsloth, including `trl`, `peft`, `accelerate`, and `bitsandandbytes`.

In [3]:
!pip install unsloth trl peft accelerate bitsandbytes

# GPU Check

This cell checks for CUDA availability and identifies the GPU being used, which is important for ensuring that the model training process can leverage GPU acceleration.

In [4]:
# For GPU check
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

CUDA available: True
GPU: NVIDIA A100-SXM4-40GB


# Load Pre-trained Model and Tokenizer

This cell loads a pre-trained language model and its corresponding tokenizer using Unsloth's `FastLanguageModel`. It specifies the model name, maximum sequence length, and enables 4-bit loading for memory efficiency.

In [5]:
from unsloth import FastLanguageModel
import torch

model_name = "unsloth/Phi-3-mini-4k-instruct-bnb-4bit"
max_seq_length = 2048  # Choose sequence length
dtype = None  # Auto detection

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.9.11: Fast Mistral patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/458 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

# Format Data

This cell defines a function `format_prompt` to structure the data from the JSON file into a format suitable for training. It then applies this function to the loaded data and creates a Hugging Face `Dataset` object.

In [6]:
from datasets import Dataset

def format_prompt(example):
    return f"### Input: {example['input']}\n### Output: {json.dumps(example['output'])}<|endoftext|>"

formatted_data = [format_prompt(item) for item in file]
dataset = Dataset.from_dict({"text": formatted_data})

# Add LoRA Adapters

This cell adds Low-Rank Adaptation (LoRA) adapters to the loaded model. LoRA is a parameter-efficient fine-tuning technique that reduces the number of trainable parameters, making fine-tuning more efficient.

In [7]:
# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=64,  # LoRA rank - higher = more capacity, more memory
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=128,  # LoRA scaling factor (usually 2x rank)
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",     # Supports any, but = "none" is optimized
    use_gradient_checkpointing="unsloth",  # Unsloth's optimized version
    random_state=3407,
    use_rslora=False,  # Rank stabilized LoRA
    loftq_config=None, # LoftQ
)

Unsloth 2025.9.11 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# Configure Trainer

This cell sets up the `SFTTrainer` from the `trl` library for supervised fine-tuning. It configures various training arguments, including batch size, learning rate, number of epochs, and output directory.

In [8]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Training arguments optimized for Unsloth
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,  # Effective batch size = 8
        warmup_steps=10,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=25,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="/content/outputs", # Changed output directory
        save_strategy="epoch",
        save_total_limit=2,
        dataloader_pin_memory=False,
        report_to="none", # Disable Weights & Biases logging
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/984 [00:00<?, ? examples/s]

# Train the Model

This cell initiates the training process using the configured `SFTTrainer`. It trains the model on the prepared dataset according to the specified training arguments.

In [9]:
# Train the model
trainer_stats = trainer.train()
# trainer_stats = trainer.train(resume_from_checkpoint = True)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 984 | Num Epochs = 3 | Total steps = 369
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 119,537,664 of 3,940,617,216 (3.03% trained)


Step,Training Loss
25,1.541000
50,1.134500
75,1.084100
100,1.057800
125,0.957400
150,0.664600
175,0.674600
200,0.634500
225,0.623300
250,0.534100


# Test the fine-tuned model with user input

This cell is a markdown header indicating the start of the model testing section.

In [14]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# Get user input
user_input = input("Enter your prompt: ")

# Test prompt
messages = [
    {"role": "user", "content": user_input},
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

# Generate attention mask
attention_mask = inputs.new_ones(inputs.shape, dtype=torch.long).to("cuda")

# Generate response
outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=256,
    use_cache=True,
    temperature=0.7,
    do_sample=True,
    top_p=0.9,
    attention_mask=attention_mask,
)

# Decode and print
response = tokenizer.batch_decode(outputs)[0]

# Find the parts of the string
user_start = response.find("<|user|>") + len("<|user|>")
user_end = response.find("<|end|>")
assistant_start = response.find("<|assistant|>") + len("<|assistant|>")
assistant_end = response.find('<|endoftext|>')

# Extract the input and output
input_text = response[user_start:user_end].strip()
output_text = response[assistant_start:assistant_end].strip().strip('"')

# Format and print
print(f"Output: {output_text}")

Enter your prompt: Though ubiquitous in late-night advertisements and seemingly a cure-all, the dietary supplement's claims are often specious
Output: Though ubiquitous in late-night advertisements and seemingly a cure-all, the dietary supplement's claims are often false.


In [15]:
# Save the model
model.save_pretrained("/content/outputs/model")

# Download the Model

This cell compresses the saved model directory into a zip file, making it easier to download. After running this cell, you can download the `model.zip` file from the Colab file browser (the folder icon on the left sidebar).

In [19]:
# Zip the model directory
!zip -r /content/outputs/model.zip /content/outputs/model

updating: content/outputs/model/ (stored 0%)
updating: content/outputs/model/adapter_model.safetensors (deflated 8%)
updating: content/outputs/model/README.md (deflated 65%)
updating: content/outputs/model/adapter_config.json (deflated 56%)
